In [17]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [18]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [19]:
def search_pay_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    param_dia_ini = datetime(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    return param_dia_ini.strftime('%Y-%m-%d'), param_dia_fin.strftime('%Y-%m-%d')

In [20]:
def api_call_pay_acc_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    param_dia_ini,  param_dia_fin = search_pay

    search_pay_acc = [
        "&", "&",
            ("partner_type", "=", "customer"),
            ("is_internal_transfer", "=", False),
        "&",
            ("date", ">=", param_dia_ini),
            ("date", "<=", param_dia_fin),
        ]


    pay_acc_fields = [
                    'name',
                    'date',
                    'amount_total',
                    'reconciled_invoice_ids',
                    'pos_session_id'
                    ]

    pay_acc_ids = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'search', [search_pay_acc])
    pay_acc_json = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'read', [pay_acc_ids], {'fields': pay_acc_fields})

    

    return pay_acc_json

In [21]:
def pay_acc_df_func(pay_acc_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_acc = []
    data_fact_ids = []

    for pay in pay_acc_json:
        if not pay['pos_session_id']:
            new = {}
            new['id'] = pay['id']
            new['name'] = pay['name']
            new['date'] = pay['date']
            new['amount_total'] = pay['amount_total']
            new['pay_fact_docs'] = pay['reconciled_invoice_ids'] if pay['reconciled_invoice_ids'] else pd.NA
            
            data_fact_ids += pay['reconciled_invoice_ids']

            data_pay_acc.append(new)

    pay_acc_df = pd.DataFrame(data_pay_acc)


    pay_acc_df['date'] = pd.to_datetime(pay_acc_df['date'], format='%Y-%m-%d')
    
    return pay_acc_df, data_fact_ids

In [22]:
def api_call_pay_pos_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    param_dia_ini,  param_dia_fin = search_pay

    search_pay_pos = [
        "&",
            ("payment_date", ">=", param_dia_ini),
            ("payment_date", "<=", param_dia_fin),
        ]


    pay_pos_fields = [
                    'pos_order_id',
                    'payment_date',
                    'amount',
                    'payment_method_id'
                    ]

    pay_pos_ids = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'search', [search_pay_pos])
    pay_pos_json = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'read', [pay_pos_ids], {'fields': pay_pos_fields})

    

    return pay_pos_json

In [23]:
def pay_pos_df_func(pay_pos_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_pos = []
    data_pos_ids = set()

    for pay in pay_pos_json:
        if pay['payment_method_id'][0] != 6:
            new = {}
            new['pay_pos_id'] = pay['id']
            new['pos_order_id'] = pay['pos_order_id'][0]
            new['pos_order_name'] = pay['pos_order_id'][1]
            new['date'] = pay['payment_date']
            new['amount'] = pay['amount']
        
            data_pos_ids.add(pay['pos_order_id'][0])
            data_pay_pos.append(new)

    pay_pos_df1 = pd.DataFrame(data_pay_pos)
    pay_pos_df = pay_pos_df1.groupby('pos_order_id').agg({'pay_pos_id':['first'], 'pos_order_name':['first'], 'date':['first'], 'amount':['sum']}).reset_index().iloc[:,[1,0,2,3,4]]
    pay_pos_df.columns = ['pay_pos_id', 'pos_order_id', 'pos_order_name', 'date', 'amount']

    pay_pos_df['date'] = pd.to_datetime(pay_pos_df['date'], format='%Y-%m-%d %H:%M:%S')
    
    return pay_pos_df, list(data_pos_ids)

In [24]:
def api_call_pos_order_func(api_params: dict, data_pos_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    pos_order_fields = [
                    'account_move',
                    'partner_id',
                    'session_move_id',
                    'amount_total'
                    ]

    pos_order_json = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [data_pos_ids], {'fields': pos_order_fields})

    return pos_order_json

In [25]:
def pos_order_func(pos_order_json: list[dict], data_fact_ids: list[int]) -> list[list[int]]:
    
    data_session_move_ids = [] 
    data_partner_ids = [] 

    for pos in pos_order_json:
        
        if pos['account_move']:
            data_fact_ids.append(pos['account_move'][0])
        
        data_partner_ids.append(pos['partner_id'][0])
        data_session_move_ids.append(pos['session_move_id'][0])

    
    return data_fact_ids, data_session_move_ids, data_partner_ids

In [26]:
def api_call_account_line_func(api_params: dict, data_session_move_ids: list[int], data_partner_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    account_line_sessions_fields = [
                    'matching_number'
                    ]

    account_line_sessions_ids = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'search', [["&", ("move_id", "in", data_session_move_ids), ("partner_id", "in", data_partner_ids)]])
    account_line_sessions_json = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'read', [account_line_sessions_ids], {'fields': account_line_sessions_fields})


    data_matching_number = [] 

    for pos in account_line_sessions_json:
        if pos['matching_number']:
            data_matching_number.append(pos['matching_number'])

    
    account_line_matching_number_fields = [
                    'move_id'
                    ]

    account_line_matching_number_ids = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'search', [['&', ("move_type", "=", "out_invoice"), ("matching_number", "in", data_matching_number)]])
    account_line_matching_number_json = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'read', [account_line_matching_number_ids], {'fields': account_line_matching_number_fields})


    return account_line_matching_number_json

In [27]:
def data_fact_ids_func(account_line_json: list[dict], data_fact_ids: list[int]) -> list[list[int]]:
    
    for line in account_line_json:
        data_fact_ids.append(line['move_id'][0])

    return list(set(data_fact_ids))

In [28]:
def api_call_acccount_docs_func(api_params: dict, data_fact_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    data_fact_fields = [
                    'name',
                    'partner_id',
                    'date',
                    'invoice_payments_widget',
                    'amount_total',
                    'amount_residual'
                    ]

    fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [data_fact_ids], {'fields': data_fact_fields})

    return fact_doc_json

In [29]:
# def pay_pos_df_func(pay_pos_json: list[dict]) -> list[pd.DataFrame, int]:
    
#     data_pay_pos = []
#     data_pos_ids = set()

#     for pay in pay_pos_json:
#         if pay['payment_method_id'][0] != 6:
#             new = {}
#             new['pay_pos_id'] = pay['id']
#             new['pos_order_id'] = pay['pos_order_id'][0]
#             new['pos_order_name'] = pay['pos_order_id'][1]
#             new['date'] = pay['payment_date']
#             new['amount'] = pay['amount']
        
#             data_pos_ids.add(pay['pos_order_id'][0])
#             data_pay_pos.append(new)

#     pay_pos_df1 = pd.DataFrame(data_pay_pos)
#     pay_pos_df = pay_pos_df1.groupby('pos_order_id').agg({'pay_pos_id':['first'], 'pos_order_name':['first'], 'date':['first'], 'amount':['sum']}).reset_index().iloc[:,[1,0,2,3,4]]
#     pay_pos_df.columns = ['pay_pos_id', 'pos_order_id', 'pos_order_name', 'date', 'amount']

#     pay_pos_df['date'] = pd.to_datetime(pay_pos_df['date'], format='%Y-%m-%d %H:%M:%S')
    
#     return pay_pos_df, list(data_pos_ids)

In [30]:
api_params = api_params_func(True)
search_pay = search_pay_func(4)

pay_acc_json = api_call_pay_acc_func(api_params, search_pay)
pay_acc_df, data_fact_ids3 = pay_acc_df_func(pay_acc_json)

pay_pos_json = api_call_pay_pos_func(api_params, search_pay)
pay_pos_df, data_pos_ids = pay_pos_df_func(pay_pos_json)

pos_order_json = api_call_pos_order_func(api_params, data_pos_ids)
data_fact_ids2, data_session_move_ids, data_partner_ids = pos_order_func(pos_order_json, data_fact_ids3)

account_line_json = api_call_account_line_func(api_params, data_session_move_ids, data_partner_ids)
data_fact_ids = data_fact_ids_func(account_line_json, data_fact_ids2)

fact_doc_json = api_call_acccount_docs_func(api_params, data_fact_ids)

In [31]:
data_pay_fact = []

for fact in fact_doc_json:
    new = {}
    new['fac_doc_id'] = fact['id']
    new['fac_doc_name'] = fact['name']
    new['fac_doc_cliente'] = fact['partner_id'][1]
    new['fac_doc_date'] = fact['date']
    new['fac_doc_total'] = fact['amount_total']
    new['fac_doc_deuda'] = fact['amount_residual']
    
    if fact['invoice_payments_widget']:
        i = 1
        for pay in fact['invoice_payments_widget']['content']:
            new[f'pay_{i}_amount'] = pay['amount']
            new[f'pay_{i}_date'] = pay['date']
            new[f'pay_{i}_journal'] = pay['journal_name']
            new[f'pay_{i}_pos'] = pay['pos_payment_name'] if pay['pos_payment_name'] else pd.NA
            i += 1

        data_pay_fact.append(new)

pay_fact_df = pd.DataFrame(data_pay_fact)

for col in pay_fact_df.columns:
    if col[-4:] == 'date':
        pay_fact_df[col] = pd.to_datetime(pay_fact_df[col], format='%Y-%m-%d')

In [32]:
pay_fact_df.loc[(pay_fact_df['pay_1_date'].dt.month.isin([1,2,3])) | (pay_fact_df['pay_2_date'].dt.month.isin([1,2,3])) | (pay_fact_df['pay_3_date'].dt.month.isin([1,2,3]))]

,fac_doc_id,fac_doc_name,fac_doc_cliente,fac_doc_date,fac_doc_total,fac_doc_deuda,pay_1_amount,pay_1_date,pay_1_journal,pay_1_pos,pay_2_amount,pay_2_date,pay_2_journal,pay_2_pos,pay_3_amount,pay_3_date,pay_3_journal,pay_3_pos
21,8286,F1-CC/2024/00820,GUADALUPE (SR.PINACHO),2024-01-17,14176.85,0.0,3703.40,2024-02-10,Punto De Venta A1,<NA>,10473.45,2024-02-10,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN
24,190,F1-CC/2024/00012,GUADALUPE (SR.PINACHO),2024-01-02,430.48,0.0,430.48,2024-02-10,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
78,368,F1-CC/2024/00029,Carlos Humberto Briones Contreras,2024-01-02,4971.11,0.0,706.26,2024-04-12,Punto De Venta A1,<NA>,4264.85,2024-03-05,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN
241,9409,F1-CC/2024/00925,GUADALUPE (SR.PINACHO),2024-01-19,505.94,0.0,505.94,2024-02-10,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
426,59391,F1-CC/2024/05291,DMADERA BCS,2024-04-11,531.30,0.0,437.73,2024-03-27,Punto De Venta A1,Tarjeta Débito A1,93.57,2024-04-11,Punto De Venta A1,Tarjeta Débito A1,NaN,NaT,NaN,NaN
468,10512,F1-CC/2024/01037,GUADALUPE (SR.PINACHO),2024-01-22,1397.31,0.0,1397.31,2024-02-10,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
539,59969,F2-CC/2024/07069,WAHOO,2024-04-11,426.61,0.0,0.54,2024-04-11,Punto De Venta A2,Efectivo A2,426.07,2024-03-23,Facturas de cliente A2,<NA>,NaN,NaT,NaN,NaN
601,2956,F1-CC/2024/00283,GUADALUPE (SR.PINACHO),2024-01-08,551.65,0.0,551.65,2024-02-10,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
615,11270,F1-CC/2024/01102,GUADALUPE (SR.PINACHO),2024-01-23,593.79,0.0,593.79,2024-02-10,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
800,11822,F1-CC/2024/01156,GUADALUPE (SR.PINACHO),2024-01-24,1053.01,0.0,1053.01,2024-02-10,Punto De Venta A1,<NA>,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN


## Pruebas